In [1]:
from paz.backend.image import load_image, show_image
from paz.applications import HaarCascadeFrontalFace, MiniXceptionFER
import paz.processors as pr
from paz.backend.image.image import crop_image
from paz.processors.detection import CropBoxes2D
import numpy as np
from pathlib import Path
from paz.abstract import SequentialProcessor
from retinaface import RetinaFace
from paz.abstract.messages import Box2D

In [8]:
class process_front_images():
    def __init__(self):
        self.folder_path = Path('EECE571L\Driver_emotion_detection\DMD\s3_rgb_face').__str__()
        self.result = np.array([]) # 2D array with element [frame_number, emotion]
        
    def get_emotion(self):
        for frame_number in range(1, 5):
            path = self.folder_path + '\%(number)06d.png' % {"number": frame_number}
            image = load_image(path)
    
            print('--------Frame {fnumber}------'.format(fnumber = frame_number))
            crop = CropBoxes2D()
            face = RetinaFace.detect_faces(path)
            coordinates = face['face_1']['facial_area']
            boxes2D = [Box2D(coordinates, score=0)]
            print('Boxes: ', boxes2D, type(boxes2D))
            if (len(boxes2D) == 0):
                continue
                
            elif (len(boxes2D) >= 2):
                areas = np.array([])
                for box in boxes2D:
                    area = (box.coordinates[2]-box.coordinates[0]) * (box.coordinates[3]-box.coordinates[1])
                    areas = np.append(areas, area)
                indices_excluding_max = np.where(areas != np.amax(areas))[0]
                print('Indices to remove', indices_excluding_max)
                for i in indices_excluding_max:
                    boxes2D = np.delete(boxes2D, i)
                print('After removing smaller boxes: ', boxes2D)            
            
            cropped_images = crop(image, boxes2D)
            show_image(cropped_images[0])
            classify = MiniXceptionFER()
            emotion = classify(cropped_images[0])['class_name']
            print([frame_number, emotion])
            self.result = np.append(self.result, [frame_number, emotion])
        print('----------------Done getting emotion------------------')
        return self.result
    
front_image_processor = process_front_images()
emotion_result = front_image_processor.get_emotion()
print('Front face emotion labeling: ', emotion_result)

--------Frame 1------
Boxes:  [Box2D(495, 302, 708, 553, 0, None)] <class 'list'>
[1, 'sad']
--------Frame 2------
Boxes:  [Box2D(494, 303, 708, 554, 0, None)] <class 'list'>
[2, 'sad']
--------Frame 3------
Boxes:  [Box2D(494, 304, 707, 557, 0, None)] <class 'list'>
[3, 'sad']
--------Frame 4------
Boxes:  [Box2D(497, 302, 715, 566, 0, None)] <class 'list'>
[4, 'sad']
----------------Done getting emotion------------------
Front face emotion labeling:  ['1' 'sad' '2' 'sad' '3' 'sad' '4' 'sad']


In [5]:
class process_side_images():
    def __init__(self):
        self.folder_path = Path('EECE571L\Driver_emotion_detection\DMD\s3_rgb_body').__str__()
        self.result = np.array([0, 0]) # path format?
    def get_face_image(self):
        for frame_number in range(1, 5):
            path = self.folder_path + '\%(number)06d.png' % {"number": frame_number}
            image = load_image(path)
    
            print('--------Frame {fnumber}------'.format(fnumber = frame_number))
            crop = CropBoxes2D()
            face = RetinaFace.detect_faces(path)
            coordinates = face['face_1']['facial_area']
            boxes2D = [Box2D(coordinates, score=0)]
            print('Boxes: ', boxes2D, type(boxes2D))
            if (len(boxes2D) == 0):
                continue
                
            elif (len(boxes2D) >= 2):
                areas = np.array([])
                for box in boxes2D:
                    area = (box.coordinates[2]-box.coordinates[0]) * (box.coordinates[3]-box.coordinates[1])
                    areas = np.append(areas, area)
                indices_excluding_max = np.where(areas != np.amax(areas))[0]
                print('Indices to remove', indices_excluding_max)
                for i in indices_excluding_max:
                    boxes2D = np.delete(boxe
                                        s2D, i)
                print('After removing smaller boxes: ', boxes2D)            
            
            cropped_images = crop(image, boxes2D)
            show_image(cropped_images[0])

side_processor = process_side_images()
side_processor.get_face_image()

--------Frame 1------
Boxes:  [Box2D(393, 25, 541, 238, 0, None)] <class 'list'>
--------Frame 2------
Boxes:  [Box2D(395, 28, 543, 238, 0, None)] <class 'list'>
--------Frame 3------
Boxes:  [Box2D(397, 28, 543, 239, 0, None)] <class 'list'>
--------Frame 4------
Boxes:  [Box2D(398, 30, 543, 240, 0, None)] <class 'list'>


### Code for detecting faces using PAZ

Poor detection performance
    4 out of 5 images were detected wrongly - boxes not appears around the face

In [ ]:
path = self.folder_path + '\%(number)06d.png' % {"number": frame_number}
            image = load_image(path)
    
            print('--------Frame {fnumber}------'.format(fnumber = frame_number))
            detect = HaarCascadeFrontalFace(draw=False)
            crop = CropBoxes2D()
            inference = detect(image)
            boxes2D = inference['boxes2D']
            print('Boxes: ', boxes2D, type(boxes2D))